In [1]:
# https://notebooks.azure.com/azureml/projects/azureml-getting-started/html/02.deploy-web-service.ipynb
from azureml.core import Workspace, Experiment, Run
import math, random, pickle, json

In [2]:
ws = Workspace.from_config()

In [3]:
# retrieve the latest run from your experiment
experiment_name = "my-first-experiment"
run = list(Experiment(workspace = ws, name = experiment_name).get_runs())[0]

In [4]:
# In the previous example you saved a file containing the pi value into run history. 
# Registering the file makes it into a model that is tracked by Azure ML model management
model = run.register_model(model_name = "pi_estimate", model_path = "outputs/pi_estimate.txt")

In [10]:
%%writefile score.py

# Create a 2-part scoring script that computes an area of a circle, given the estimate within the pi_estimate model:
#
# 1.  The init method that loads the model. You can retrieve registered model using Model.get_model_path method
# 2.  The run method that gets invoked when you call the web service. It computes the area of a circle using 
#     area=π∗radius^2. The inputs and outputs are passed as json-formatted strings
import pickle, json
from azureml.core.model import Model

def init():
    global pi_estimate
    model_path = Model.get_model_path(model_name = "pi_estimate")
    with open(model_path, "rb") as f:
        pi_estimate = float(pickle.load(f))

def run(raw_data):
    try:
        radius = json.loads(raw_data)["radius"]
        result = pi_estimate * radius**2
        return json.dumps({"area": result})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Writing score.py


In [5]:
# specify the library dependencies of your scoring script as conda yml file. 
# This example doesn't use any special libraries, so let's simply use Azure ML's default dependencies
from azureml.core.conda_dependencies import CondaDependencies 

cd = CondaDependencies()
cd.save_to_file(".", "myenv.yml")

'myenv.yml'

In [6]:
# Deploy the web service on Azure Container Instance: a serverless compute for running Docker images.  Azure ML service takes 
# care of packaging your model, scoring script and dependencies into Docker image and deploying it
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage

# Define the configuration of compute: ACI with 1 cpu core and 1 gb of memory.
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

# Specify image configuration: scoring script, Python runtime (PySpark is other option), and conda file of library dependencies
image_config = ContainerImage.image_configuration(execution_script = "score.py", 
                                    runtime = "python", 
                                    conda_file = "myenv.yml")

# Deploy the web service as an image containing the registered model.
service = Webservice.deploy_from_model(name = "area-calculator",
                                       deployment_config = aci_config,
                                       models = [model],
                                       image_config = image_config,
                                       workspace = ws)

# The service deployment can take several minutes: wait for completion.
service.wait_for_deployment(show_output = True)

Creating image
Running...................................
Succeeded
Image creation operation finished for image area-calculator:2, operation "Succeeded"
Creating service
Running..............
SucceededACI service creation operation finished, operation "Succeeded"


In [7]:
from ipywidgets import interact

def get_area(radius):
    request = json.dumps({"radius": radius})
    response = service.run(input_data = request)
    return json.loads(response)["area"]

interact(get_area,radius=(0,10))

A Jupyter Widget

<function __main__.get_area>

In [8]:
service.delete()